# 準備

In [1]:
# 必要ライブラリのインポート
%matplotlib inline
import numpy as np
import os
import re
import matplotlib.pyplot as plt
import japanize_matplotlib
from IPython.display import display
from PIL import Image
import pandas as pd

from sklearn.model_selection import train_test_split

# torch関連
import torch
import torch.nn as nn
import torch.optim as optim
from torchinfo import summary
from torchviz import make_dot
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

from dataset import CustomImageDataset
from utils import fit, evaluate_history, torch_seed
from models.rdn import RdnDenseNet
from models.Densenet import DenseNet

In [2]:
# warning表示off
import warnings
warnings.simplefilter('ignore')

# デフォルトフォントサイズ変更
plt.rcParams['font.size'] = 14

# デフォルトグラフサイズ変更
plt.rcParams['figure.figsize'] = (6,6)

# デフォルトで方眼表示ON
plt.rcParams['axes.grid'] = True

# numpyの表示桁数設定
np.set_printoptions(suppress=True, precision=5)

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


# データ読み込み

In [4]:
df = pd.read_csv("../data/tiny-imagenet-200/val/val_annotations.txt", sep=r'\s+', header=None)
print('クラスの数: ', df[1].nunique())
print('ファイル名からクラス名を出力: ', df[df[0] == 'val_1.JPEG'][1].iloc[0])

クラスの数:  200
ファイル名からクラス名を出力:  n04067472


In [5]:
# パスを指定
train_data_root = '../data/tiny-imagenet-200/train'
test_data_root = '../data/tiny-imagenet-200/val'
submission_data = '../data/tiny-imagenet-200/test'

# transformの定義：train用
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])
# transformの定義：val, test, submission用
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# クラスインデックスを作成
pattern = re.compile(r'n[0-9]{8}')
classes = sorted(filter(pattern.fullmatch, os.listdir(train_data_root)))
class_idx = {cls_name: idx for idx, cls_name in enumerate(classes)}

# データの読み込み
full_dataset = CustomImageDataset(root_dir=train_data_root, mode='full', class_idx=class_idx, transform=transform_train)
train_data, val_data = train_test_split(full_dataset.data, test_size=0.2, random_state=42)

train_set = CustomImageDataset(root_dir=train_data_root, mode='train', class_idx=class_idx, transform=transform_train, load_data=train_data)
val_set = CustomImageDataset(root_dir=train_data_root, mode='val', class_idx=class_idx, transform=transform_test, load_data=val_data)
test_set = CustomImageDataset(root_dir=test_data_root, mode='test', class_idx=class_idx, transform=transform_test)

# ミニバッチのサイズ指定
batch_size = 100

# 訓練データローダー
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [6]:
"""
# グレースケール確認
grayscale_indices = []

for idx in range(len(train_set)):
    img_path, _ = train_set.data[idx]
    image = Image.open(img_path)

    # 画像がグレースケールか確認
    if image.mode != 'RGB':
        grayscale_indices.append(idx)

print(f"グレースケール画像: {grayscale_indices}")
"""

'\n# グレースケール確認\ngrayscale_indices = []\n\nfor idx in range(len(train_set)):\n    img_path, _ = train_set.data[idx]\n    image = Image.open(img_path)\n\n    # 画像がグレースケールか確認\n    if image.mode != \'RGB\':\n        grayscale_indices.append(idx)\n\nprint(f"グレースケール画像: {grayscale_indices}")\n'

In [7]:
"""
# データセット確認
image_check, label_check = train_set[0]
print("イメージサイズ: ", image_check.shape)
print("正解ラベル: ", label_check)

plt.figure(figsize=(2,3))
plt.title("画像")
# (image * 0.5 + 0.5)：正規化を元に戻す。.permute(1,2,0)：高さ、幅、チャンネルの順にしてあげる
plt.imshow((image_check * 0.5 + 0.5).permute(1,2,0).numpy())
plt.axis('off')
"""

'\n# データセット確認\nimage_check, label_check = train_set[0]\nprint("イメージサイズ: ", image_check.shape)\nprint("正解ラベル: ", label_check)\n\nplt.figure(figsize=(2,3))\nplt.title("画像")\n# (image * 0.5 + 0.5)：正規化を元に戻す。.permute(1,2,0)：高さ、幅、チャンネルの順にしてあげる\nplt.imshow((image_check * 0.5 + 0.5).permute(1,2,0).numpy())\nplt.axis(\'off\')\n'

In [8]:
"""
# train_loaderを確認（画像のshapeが揃っているかの確認も兼ねている）
for image_check, label_check in train_loader:
    print(image_check.shape)
    print(label_check)
    print("-" * 30)

for image_check, label_check in val_loader:
    print(image_check.shape)
    print(label_check)
    print("-" * 30)
"""

'\n# train_loaderを確認（画像のshapeが揃っているかの確認も兼ねている）\nfor image_check, label_check in train_loader:\n    print(image_check.shape)\n    print(label_check)\n    print("-" * 30)\n\nfor image_check, label_check in val_loader:\n    print(image_check.shape)\n    print(label_check)\n    print("-" * 30)\n'

# モデル出力

In [9]:
net = DenseNet().to(device)
criterion = nn.CrossEntropyLoss()
lr=0.1
optimizer = optim.Adam(net.parameters(), lr=lr)

In [10]:
summary(net,depth=4)

Layer (type:depth-idx)                   Param #
DenseNet                                 --
├─Sequential: 1-1                        --
│    └─Conv2d: 2-1                       9,408
│    └─BatchNorm2d: 2-2                  128
│    └─ReLU: 2-3                         --
│    └─MaxPool2d: 2-4                    --
│    └─_DenseBlock: 2-5                  --
│    │    └─_DenseLayer: 3-1             --
│    │    │    └─BatchNorm2d: 4-1        128
│    │    │    └─ReLU: 4-2               --
│    │    │    └─Conv2d: 4-3             8,192
│    │    │    └─BatchNorm2d: 4-4        256
│    │    │    └─Conv2d: 4-5             36,864
│    │    │    └─Dropout: 4-6            --
│    │    └─_DenseLayer: 3-2             --
│    │    │    └─BatchNorm2d: 4-7        192
│    │    │    └─ReLU: 4-8               --
│    │    │    └─Conv2d: 4-9             12,288
│    │    │    └─BatchNorm2d: 4-10       256
│    │    │    └─Conv2d: 4-11            36,864
│    │    │    └─Dropout: 4-12           --
│   

In [11]:
def eval_loss(loader, device, net, criterion):

    # データローダーから最初の1セットを取得する
    for images, labels in loader:
        break

    # デバイスの割り当て
    inputs = images.to(device)
    labels = labels.to(device)

    # 予測計算
    outputs = net(inputs)

    #  損失計算
    loss = criterion(outputs, labels)

    return loss

loss = eval_loss(test_loader, device, net, criterion)
g = make_dot(loss, params=dict(net.named_parameters()))
display(g)

KeyboardInterrupt: 